In [43]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score

In [2]:
actual = pd.read_csv('values_ml_papers.csv')

In [3]:
all_responses = [pd.read_csv('responses/' + f, usecols=['Title', 'response']) for f in os.listdir('responses') if 'csv' in f]
df = pd.concat(all_responses)

In [4]:
imploded = df.groupby('Title')['response'].apply(list).reset_index()

In [11]:
results = pd.DataFrame(imploded['response'].to_list(), columns=actual.loc[:,'Novelty':'Fast'].columns).applymap(lambda x: x.lower() if isinstance(x, str) else x)

/var/folders/36/6lk0rs9x1r7chcsmnk80n6580000gn/T/ipykernel_86728/1706142083.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  results = pd.DataFrame(imploded['response'].to_list(), columns=actual.loc[:,'Novelty':'Fast'].columns).applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [37]:
results = results.replace({'yes': 1, 'no': 0, 'yes.': 1, 'no.': 0})
results = results.map(lambda v: 0 if isinstance(v, str) and ('does not' in v or 'unanswerable' in v) else v)
results = results.map(lambda v: 1 if isinstance(v, str) and 'yes' in v else v)
results = results.map(lambda v: 0 if isinstance(v, str) else v)

In [53]:
actual.Fairness[results.Fairness.dropna().index].any()

False

In [54]:
results.Fairness.any()

False

In [56]:
confusion_matrix(actual.Fairness[results.Fairness.dropna().index], results.Fairness.dropna()).ravel()

/Users/vyoma/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


array([70])

In [100]:
def calculate_metrics(true_labels, predicted_labels):
    true_ls = true_labels[predicted_labels.dropna().index]
    pred_ls = predicted_labels.dropna()

    tn = ((true_ls == 0) & (pred_ls == 0)).sum()
    tp = ((true_ls == 1) & (pred_ls == 1)).sum()
    fp = ((true_ls == 0) & (pred_ls == 1)).sum()
    fn = ((true_ls == 1) & (pred_ls == 0)).sum()
    print('True Positives:', tp, 'True Negatives:', tn, 'False Positives:', fp, 'False Negatives:', fn)
    
    fpr = fp / (fp + tn) if (fp + tn) > 0 else None  # None indicates not applicable or undefined
    fnr = fn / (tp + fn) if (tp + fn) > 0 else None
    
    f1 = f1_score(true_ls, pred_ls, zero_division=1)
    
    return {"FPR": fpr, "FNR": fnr, "F1 Score": f1}

In [101]:
calculate_metrics(actual.Novelty, results.Novelty)

True Positives: 50 True Negatives: 0 False Positives: 17 False Negatives: 2


{'FPR': 1.0, 'FNR': 0.038461538461538464, 'F1 Score': 0.8403361344537815}

In [102]:
calculate_metrics(actual.Fairness, results.Fairness)

True Positives: 0 True Negatives: 70 False Positives: 0 False Negatives: 0


{'FPR': 0.0, 'FNR': None, 'F1 Score': 1.0}

In [103]:
calculate_metrics(actual.Performance, results.Performance)

True Positives: 49 True Negatives: 6 False Positives: 7 False Negatives: 8


{'FPR': 0.5384615384615384,
 'FNR': 0.14035087719298245,
 'F1 Score': 0.8672566371681416}

In [106]:
calculate_metrics(actual.Reproducibility, results.Reproducibility)

True Positives: 1 True Negatives: 30 False Positives: 39 False Negatives: 0


{'FPR': 0.5652173913043478, 'FNR': 0.0, 'F1 Score': 0.04878048780487805}

In [109]:
calculate_metrics(actual['Easy to implement'], results['Easy to implement'])

True Positives: 4 True Negatives: 37 False Positives: 28 False Negatives: 1


{'FPR': 0.4307692307692308, 'FNR': 0.2, 'F1 Score': 0.21621621621621623}

In [110]:
calculate_metrics(actual['Building on classic work'], results['Building on classic work'])

True Positives: 10 True Negatives: 27 False Positives: 22 False Negatives: 11


{'FPR': 0.4489795918367347,
 'FNR': 0.5238095238095238,
 'F1 Score': 0.37735849056603776}

In [111]:
calculate_metrics(actual['Building on recent work'], results['Building on recent work'])

True Positives: 53 True Negatives: 1 False Positives: 14 False Negatives: 3


{'FPR': 0.9333333333333333,
 'FNR': 0.05357142857142857,
 'F1 Score': 0.8617886178861789}